In [1]:
import sys
sys.path.append('/Users/atsky/work/catboost/catboost/python-package')

In [2]:
import catboost_dev

In [3]:
import catboost

In [4]:
import numpy as np
import pandas as pd

In [5]:
from catboost.utils import eval_metric

## Train model on real data

In [6]:
train = pd.read_csv('train.csv')

In [7]:
train.head()

,event,hitTime,pmtID,pmt_x,pmt_y,pmt_z,E,R,x,y,z
0,0,249.992610,14175,-5411.8320,16838.455,-7471.1797,4.747791,14610.378,8290.779,11995.618,911.74286
1,0,40.010310,17319,12080.4190,14903.949,-758.8111,4.747791,14610.378,8290.779,11995.618,911.74286
2,0,162.123200,16882,4993.4946,18464.752,-1660.7045,4.747791,14610.378,8290.779,11995.618,911.74286
3,0,51.875614,14951,15409.2930,9932.035,-5705.1180,4.747791,14610.378,8290.779,11995.618,911.74286
4,0,79.817500,10947,5650.6810,11274.457,-14477.4450,4.747791,14610.378,8290.779,11995.618,911.74286


In [8]:
result = []

for name, group in train.groupby("event"):
    result.append({
        "evtID" : name, 
        "E": group['E'].mean(),
       # "R": group['R'].mean(),
        "N": len(group),
        "m_x": group['pmt_x'].mean(),
        "m_y": group['pmt_y'].mean(),
        "m_z": group['pmt_z'].mean(),
        "m_time" : group['hitTime'].mean() - group['hitTime'].min()
    })

In [9]:
train_df = pd.DataFrame.from_records(result)

In [10]:
train_df.head()

,E,N,evtID,m_time,m_x,m_y,m_z
0,4.747791,8168,0,71.075894,6761.406566,9939.232009,782.970216
1,3.919721,6534,1,74.862666,9153.639035,4241.897838,-5684.371157
2,6.823932,11342,2,72.518375,11446.942855,-3286.644500,203.028891
3,3.765940,6031,3,88.112089,-1474.260245,-11511.730835,2415.198830
4,3.217473,5266,4,72.440035,-7096.045951,697.188637,7900.703944


In [11]:
from catboost import Pool, CatBoostRegressor

In [12]:
train_pool = Pool(train_df[["N", "m_time", "m_x", "m_y", "m_z"]].values, 
                  train_df["E"].values)

In [13]:
model = CatBoostRegressor(iterations=200, 
                          depth=3, 
                          learning_rate=0.1, 
                          loss_function='RMSE')

In [14]:
model.fit(train_pool, verbose=True)

0:	learn: 5.7815453	total: 58.6ms	remaining: 11.7s
1:	learn: 5.3303107	total: 61.8ms	remaining: 6.12s
2:	learn: 4.8771166	total: 64.2ms	remaining: 4.21s
3:	learn: 4.4806985	total: 66ms	remaining: 3.23s
4:	learn: 4.1222883	total: 67.6ms	remaining: 2.64s
5:	learn: 3.8502985	total: 70.1ms	remaining: 2.27s
6:	learn: 3.5995688	total: 71.9ms	remaining: 1.98s
7:	learn: 3.3400129	total: 73.6ms	remaining: 1.77s
8:	learn: 3.0994986	total: 75.5ms	remaining: 1.6s
9:	learn: 2.8665372	total: 77.4ms	remaining: 1.47s
10:	learn: 2.6329865	total: 79.1ms	remaining: 1.36s
11:	learn: 2.4506017	total: 80.9ms	remaining: 1.27s
12:	learn: 2.2739705	total: 82.9ms	remaining: 1.19s
13:	learn: 2.0980248	total: 84.8ms	remaining: 1.13s
14:	learn: 1.9464981	total: 86.4ms	remaining: 1.06s
15:	learn: 1.8187154	total: 88.2ms	remaining: 1.01s
16:	learn: 1.7346192	total: 90.2ms	remaining: 971ms
17:	learn: 1.6192661	total: 91.8ms	remaining: 929ms
18:	learn: 1.5532228	total: 93.6ms	remaining: 892ms
19:	learn: 1.4619336	tota

187:	learn: 0.4354173	total: 427ms	remaining: 27.3ms
188:	learn: 0.4353020	total: 431ms	remaining: 25.1ms
189:	learn: 0.4349365	total: 434ms	remaining: 22.9ms
190:	learn: 0.4344874	total: 437ms	remaining: 20.6ms
191:	learn: 0.4341828	total: 440ms	remaining: 18.4ms
192:	learn: 0.4340399	total: 443ms	remaining: 16.1ms
193:	learn: 0.4338291	total: 446ms	remaining: 13.8ms
194:	learn: 0.4337551	total: 449ms	remaining: 11.5ms
195:	learn: 0.4331105	total: 453ms	remaining: 9.24ms
196:	learn: 0.4330366	total: 461ms	remaining: 7.02ms
197:	learn: 0.4329501	total: 466ms	remaining: 4.71ms
198:	learn: 0.4231237	total: 471ms	remaining: 2.37ms
199:	learn: 0.4230099	total: 476ms	remaining: 0us


In [15]:
model.eval_metrics(train_pool, 
             'Huber:delta=1.0')['Huber:delta=1'][-1]

0.07729220384018566

In [16]:
model = CatBoostRegressor(iterations=200, 
                          depth=3, 
                          learning_rate=0.1, 
                          loss_function='Huber:delta=1.0')
model.fit(train_pool, verbose=True)

0:	learn: 4.7124397	total: 2.67ms	remaining: 532ms
1:	learn: 4.2882511	total: 6.03ms	remaining: 597ms
2:	learn: 3.7783974	total: 10.1ms	remaining: 665ms
3:	learn: 3.3501606	total: 13.5ms	remaining: 660ms
4:	learn: 3.0933200	total: 22.7ms	remaining: 885ms
5:	learn: 2.9007275	total: 27.9ms	remaining: 901ms
6:	learn: 2.7033423	total: 33.5ms	remaining: 924ms
7:	learn: 2.3964673	total: 38.4ms	remaining: 922ms
8:	learn: 2.0470080	total: 41.9ms	remaining: 888ms
9:	learn: 1.8575951	total: 44.8ms	remaining: 851ms
10:	learn: 1.7490122	total: 46.6ms	remaining: 801ms
11:	learn: 1.4792989	total: 49.6ms	remaining: 777ms
12:	learn: 1.4272878	total: 53.6ms	remaining: 770ms
13:	learn: 1.3675890	total: 55.3ms	remaining: 734ms
14:	learn: 1.2747647	total: 57ms	remaining: 703ms
15:	learn: 1.0990504	total: 61ms	remaining: 702ms
16:	learn: 1.0395607	total: 63.3ms	remaining: 681ms
17:	learn: 0.7141644	total: 66ms	remaining: 667ms
18:	learn: 0.6646635	total: 67.8ms	remaining: 646ms
19:	learn: 0.6386817	total: 

163:	learn: 0.0756533	total: 426ms	remaining: 93.4ms
164:	learn: 0.0755672	total: 436ms	remaining: 92.5ms
165:	learn: 0.0749631	total: 441ms	remaining: 90.3ms
166:	learn: 0.0748757	total: 448ms	remaining: 88.4ms
167:	learn: 0.0735130	total: 451ms	remaining: 85.9ms
168:	learn: 0.0735050	total: 455ms	remaining: 83.5ms
169:	learn: 0.0733454	total: 458ms	remaining: 80.8ms
170:	learn: 0.0732778	total: 463ms	remaining: 78.4ms
171:	learn: 0.0731629	total: 466ms	remaining: 75.9ms
172:	learn: 0.0730541	total: 468ms	remaining: 73.1ms
173:	learn: 0.0729635	total: 470ms	remaining: 70.2ms
174:	learn: 0.0729601	total: 472ms	remaining: 67.4ms
175:	learn: 0.0726508	total: 474ms	remaining: 64.6ms
176:	learn: 0.0720860	total: 476ms	remaining: 61.8ms
177:	learn: 0.0707840	total: 477ms	remaining: 59ms
178:	learn: 0.0706671	total: 479ms	remaining: 56.2ms
179:	learn: 0.0705105	total: 481ms	remaining: 53.5ms
180:	learn: 0.0704656	total: 483ms	remaining: 50.7ms
181:	learn: 0.0704532	total: 485ms	remaining: 47

In [17]:
model.eval_metrics(train_pool, 
             'Huber:delta=1.0')['Huber:delta=1'][-1]

0.06808689871233206

In [18]:
def check_decreasing(x):
    for i in range(1, len(x)):
        if x[i - 1] < x[i]:
            print("Error: Sequence is not decreasing {} {}".format(x[i - 1], x[i]))

In [19]:
check_decreasing(model.eval_metrics(train_pool, 
             'Huber:delta=1.0')['Huber:delta=1'])

## Manual test

In [20]:
def huber_loss(arr, delta):
    def huber_loss_int(a, delta):
        if abs(a) <= delta:
            return 0.5 * a**2
        return 0.5 * delta**2 + delta * (abs(a) - delta)
    sum = 0
    for i in arr:
        sum += huber_loss_int(i, delta)
    return sum/len(arr)

In [21]:
huber_loss(np.abs(model.predict(train_pool) - train_df["E"].values), 1.0)

0.06808689978064958

# Test on generated data

Generate regression samples and compare RMSE loss vs Huber loss

In [22]:
import random 
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import HuberRegressor, Ridge
from sklearn.model_selection import train_test_split

In [23]:
def generate_data():
    X, y = make_regression(n_samples=1000, n_features=3, noise=1.0,
                       bias=100.0)

    X_o, y_o = make_regression(n_samples=50, n_features=3, noise=1.0,
                       bias=20.0)
    
    X = np.vstack((X, X_o))
    y = np.concatenate((y, y_o))
    return X, y

In [24]:
for i in range(50):
    X, y = generate_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    train_pool = Pool(X_train, y_train)

    delta = np.exp(np.random.randn())
    lerning_rate = random.choice([0.5, 0.2, 0.1, 0.05])

    model_0 = CatBoostRegressor(iterations=2000, 
                                depth=3, 
                                learning_rate=lerning_rate, 
                                loss_function='RMSE')
    model_0.fit(train_pool, verbose=False)


    model_1 = CatBoostRegressor(iterations=2000, 
                                depth=3, 
                                learning_rate=lerning_rate, 
                                loss_function='Huber:delta='+ str(delta))
    model_1.fit(train_pool, verbose=False)
    
    values = model_1.eval_metrics(train_pool, 
             'Huber:delta='+ str(delta))
    for v in values.values():
        check_decreasing(values.values())

    e0 = eval_metric(y_test, model_0.predict(X_test), 'Huber:delta=' + str(delta), np.ones(len(y_test)))[0]
    e1 = eval_metric(y_test, model_1.predict(X_test), 'Huber:delta=' + str(delta), np.ones(len(y_test)))[0]
    print("RMSE: {:07.4f} HUBER: {:07.4f} delta {:.6f} lr {}".format(e0, e1, delta, lerning_rate))

RMSE: 05.6816 HUBER: 06.6150 delta 0.590150 lr 0.05
RMSE: 12.7820 HUBER: 11.8960 delta 1.152457 lr 0.05
RMSE: 04.2736 HUBER: 03.4891 delta 0.297049 lr 0.1
RMSE: 11.0294 HUBER: 09.1119 delta 1.138895 lr 0.1
RMSE: 03.4595 HUBER: 04.8476 delta 0.212763 lr 0.05
RMSE: 04.5424 HUBER: 03.0634 delta 0.401317 lr 0.1
RMSE: 09.6968 HUBER: 06.7944 delta 1.165158 lr 0.2
RMSE: 19.7108 HUBER: 13.6352 delta 1.754551 lr 0.2
RMSE: 10.5491 HUBER: 08.5960 delta 0.918768 lr 0.1
RMSE: 06.6059 HUBER: 04.3350 delta 0.566111 lr 0.1
RMSE: 04.2369 HUBER: 06.7203 delta 0.337879 lr 0.05
RMSE: 09.0456 HUBER: 05.3088 delta 0.797978 lr 0.2
RMSE: 08.0210 HUBER: 08.0789 delta 0.827216 lr 0.05
RMSE: 00.2763 HUBER: 00.7180 delta 0.023978 lr 0.1
RMSE: 08.0669 HUBER: 07.2123 delta 0.544158 lr 0.2
RMSE: 02.7694 HUBER: 02.0968 delta 0.259278 lr 0.2
RMSE: 13.5174 HUBER: 07.2794 delta 0.838186 lr 0.2
RMSE: 02.4076 HUBER: 01.5924 delta 0.201997 lr 0.1
RMSE: 05.7271 HUBER: 06.3302 delta 0.462090 lr 0.05
RMSE: 29.4544 HUBER: 21.6